In [ ]:
import pandas as pd

# Charger le dataset Paris préparé
df = pd.read_csv('../data/paris_dataset_pre_ml.csv')

print("Dataset chargé avec succès.")
print(f"Dimensions : {df.shape}")

# 0 - Sélection des variables numériques des données et les standardisées

In [ ]:

import numpy as np
from sklearn.preprocessing import StandardScaler

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

var_num = df[num_cols].copy()

# Imputation de la médiane pour la valeurs manquantes 
for col in var_num.columns:
    if var_num[col].isna().any():
        var_num[col] = var_num[col].fillna(var_num[col].median())

# Standardisation
scaler = StandardScaler()
var_num_scaled = scaler.fit_transform(var_num)
var_num_scaled_df = pd.DataFrame(var_num_scaled, columns=num_cols, index=df.index)


print(" (5 premières lignes des variables numériques standardisées) :")
print(var_num_scaled_df.head())

# 1 — Scree Plot : Variance expliquée

Cette section affiche la proportion de variance expliquée par chaque composante principale et la variance cumulée.

*Objectif : déterminer combien de composantes garder (ex. 80% / 90% de variance).*

In [ ]:
# 1) Scree plot — variance expliquée en pourcentage (composante cumulée + par composante)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


n_comp = 26
print(f"Scree: calcul PCA local avec {n_comp} composantes")
_pca_scree = PCA(n_components=n_comp)
_pca_scree.fit(X_scaled_df)
# Exprimer en pourcentage
explained = _pca_scree.explained_variance_ratio_ * 100
cum = explained.cumsum()

plt.figure(figsize=(8,5))
plt.bar(range(1, len(explained)+1), explained, alpha=0.6, label='Pourcentage par PC')
plt.plot(range(1, len(explained)+1), cum, marker='o', color='red', label='Variance cumulée (%)')
plt.xlabel('Composante principale (PC)')
plt.ylabel('Pourcentage de variance expliquée (%)')
plt.title('Scree Plot — Variance expliquée par composante (%)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Indiquer le nombre de composantes pour atteindre 80% et 90% (en %)
n80 = (cum >= 80).argmax() + 1 if (cum >= 80).any() else len(cum)

print(f"Composantes nécessaires pour 80% de variance: {n80}")


Le graphique des valeurs propres montre une décroissance progressive et logarithmique de la variance expliquée, sans apparition d’un coude marqué. Cela indique que la variance est expliqué de manière diffuse entre les différentes composantes principales. Par conséquent, le choix du nombre de composantes retenues le pourcentage de variance cumulée expliquée dans notre cas on choisis de stopper a 80% soit 13 PC pour 26 variables.

# 2 — Valeurs propres (Eigenvalues)

Ici on affiche les valeurs propres et un barplot des eigenvalues pour voir la distribution de la variance en valeurs (scree en valeurs).

*Objectif : identifier les composantes ayant une variance significative.*

In [ ]:
# 2) Étude simple des valeurs propres (13 PC)
from sklearn.decomposition import PCA
import pandas as pd

n_pc = 13
pca13 = PCA(n_components=n_pc)
pca13.fit(X_scaled_df)

eigenvalues = pca13.explained_variance_
explained_pct = (pca13.explained_variance_ratio_ * 100).round(3)
cumulative_pct = explained_pct.cumsum().round(3)

ev13 = pd.DataFrame({
    'eigenvalue': eigenvalues.round(4),
    'explained_pct': explained_pct,
    'cumulative_pct': cumulative_pct
}, index=[f'PC{i+1}' for i in range(n_pc)])

print('\n--- Valeurs propres (13 PC) ---')
print(ev13)

# Sauvegarde simple
ev13.to_csv('../data/pca_ev_13_simple.csv')
print('\nSaved: ../data/pca_ev_13_simple.csv')

In [ ]:
# 3) Loadings pour PC1 — barplot des contributions (signées) et top features (sur 13 PC)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Récupère les loadings depuis la PCA utilisée ci-dessus (13 PC) ou fallback
try:
    components = _pca.components_
    pc_cols = [f'PC{i+1}' for i in range(components.shape[0])]
    loadings_df = pd.DataFrame(components.T, index=X_scaled_df.columns, columns=pc_cols)
except Exception:
    # fallback: recalcule avec 13 PC
    _pca_fallback = PCA(n_components=min(13, X_scaled_df.shape[1]))
    _pca_fallback.fit(X_scaled_df)
    loadings_df = pd.DataFrame(_pca_fallback.components_.T, index=X_scaled_df.columns, columns=[f'PC{i+1}' for i in range(_pca_fallback.components_.shape[0])])

# PC1 loadings
pc1 = loadings_df['PC1']
# Pour affichage horizontal, trier par valeur signée (négatifs d'abord)

# Afficher les top N features par importance absolue
top_n = 25
top_feats = pc1.abs().sort_values(ascending=False).head(top_n).index
# Obtenir valeurs signées pour ces features et trier pour le barh
vals = pc1.loc[top_feats].sort_values(ascending=True)

plt.figure(figsize=(8,10))
plt.barh(vals.index, vals.values, color='C1', alpha=0.8)
plt.xlabel('Loading (PC1) — contribution signée')
plt.title(f'Loadings signés pour PC1 — top {top_n} features')
plt.tight_layout()
plt.show()

print(f"\nTop {top_n} features par contribution absolue à PC1:")
print(pc1.abs().sort_values(ascending=False).head(top_n))